## Este notebook toma los datos de los autos patentados para poder relativizar los robos de marcas y modelos

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
pats = pd.read_csv('C:/Users/Ilcarbo/Documents/DATA/backup cagadas/pats_final.csv')

In [2]:
robos = pd.read_csv('./curados/para_EDA.csv')

In [6]:
#agarro el año de patentamiento de los robados
robos['año_pat'] = robos['fecha_inscripcion_inicial'].apply(lambda x: x.split('-')[0])

In [7]:
#me quedo solo con los robados que fueron patentados en 2018 y 2019
robos18_19 = robos[(robos['año_pat'] == '2018') | (robos['año_pat'] == '2019')]

In [9]:
#dropeo la columna falopa de año 20
robos18_19.drop(robos18_19[robos18_19['automotor_anio_modelo'] == 20].index, axis = 0, inplace= True)

In [10]:
robos18_19.shape

(1255, 19)

In [57]:
# conteo de robos por marca (me quedo con los primeros 10, los que tienen más de 10 robados)
marca = pd.DataFrame(robos18_19['automotor_marca_descripcion'].value_counts())[0:11]

In [70]:
# conteos de robos por modelo
modelo = pd.DataFrame(robos18_19['automotor_modelo_descripcion'].value_counts())

In [32]:
# conteos de patentamientos por marca
marca_pats = pd.DataFrame(pats['automotor_marca_descripcion'].value_counts())

In [58]:
# reseteo el indice para que marca quede como columna y cambio nombres de columnas
marca.reset_index(inplace= True)
marca.columns = ['marca', 'robados']

In [44]:
# reseteo el indice para que marca quede como columna y cambio nombres de columnas
marca_pats.reset_index(inplace = True)
marca_pats.columns = ['marca', 'robados']

In [38]:
#marca['rob_c_100m'] = (marca['robados'] * 100000) / marca_pats.loc[marca['marca']]['automotor_marca_descripcion']

In [59]:
# mergeo robados por marca con patentados por marca
marca_rel = marca.merge(marca_pats, how= 'left', on= 'marca')

In [60]:
# cambio nombre de columnas
marca_rel.columns = ('marca', 'robados', 'patentados')

In [61]:
# genero columna de robos relativos
marca_rel['rob_c_100m'] = (marca_rel['robados'] * 100000) / marca_rel['patentados']

In [65]:
marca_rel.sort_values(by= 'rob_c_100m', ascending= False, inplace = True)

In [71]:
# lo mismo con modelos
modelo_pats = pd.DataFrame(pats['automotor_modelo_descripcion'].value_counts())

In [72]:
modelo.reset_index(inplace= True)
modelo.columns = ['modelo', 'robados']
modelo_pats.reset_index(inplace= True)
modelo_pats.columns = ['modelo', 'patentados']

In [104]:
#quedarme con aquellos que tienen mas de 10 robos
modelo = modelo[modelo['robados'] >=10]

In [105]:
modelo_rel = modelo.merge(modelo_pats, how= 'left', on= 'modelo')

In [106]:
modelo_rel.columns = ('modelo', 'robados', 'patentados')

In [107]:
modelo_rel['rob_c_100m'] = (modelo_rel['robados'] * 100000) / modelo_rel['patentados']

In [108]:
modelo_rel.sort_values(by= 'rob_c_100m', ascending= False, inplace= True) # inplace = True

In [83]:
# veo recuperados patentados en 18/19
recup = robos[robos['tramite_tipo'] == 'COMUNICACIÓN DE RECUPERO']

In [85]:
recup18_19 = recup[(recup['año_pat'] == '2018') | (recup['año_pat'] == '2019')]

In [86]:
recup18_19.shape

(58, 19)

In [110]:
modelo_rel #ver bien lo de los modelos del UP (MOVE, HIGH)

,modelo,robados,patentados,rob_c_100m
32,FOX,12,3332,360.144058
28,MOVE,14,4006,349.475786
20,GOLF,24,6924,346.620451
19,BERLINGO,24,7149,335.711288
0,208,80,32160,248.756219
1,AMAROK,59,27113,217.607790
11,SURAN,31,15546,199.408208
30,HIGH,14,7408,188.984881
8,POLO,43,24570,175.010175
13,FIESTA,31,18018,172.050172


In [111]:
marca_rel.to_csv('./curados/robos_por_marca.csv', index= False)
modelo_rel.to_csv('./curados/robos_por_modelo.csv', index= False)